In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

In [2]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        arches[k]=str(i+1)+"->"+str(j+1)
        k+=1
  return NA, arches


def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

In [3]:
NN = np.array([[0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0]])

Aeq, arc_idxs = NN2NA(NN)
C = np.array([10.0, 20.0, 10.0, 10.0, 10.0, 30.0])
beq = np.array([10.0, 20.0, 15.0, -25.0, -20.0])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

np.set_printoptions(formatter={'float': lambda x: "{0:0.0f}".format(x)})
print('## Optimizer inputs ## \n'
      'Cost vector: %s \n\n '
      'Arches in order:\n%s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

## Optimizer inputs ## 
Cost vector: [10 20 10 10 10 30] 

 Arches in order:
['1->4', '1->5', '2->4', '2->5', '3->4', '3->5'] 

A_eq Node-Arc matrix:
[[1 1 0 0 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 1 1]
 [-1 0 -1 0 -1 0]
 [0 -1 0 -1 0 -1]] 

b_eq demand-supply vector: [10 20 15 -25 -20] 

Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None))


In [4]:
# OPTIMIZE:

res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

# GET THE SOLUTION:

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print('## Results ##')
df=pd.DataFrame(res.x,index=arc_idxs,columns=['Quantity'])
print(df)

## Results ##
      Quantity
1->4      10.0
1->5       0.0
2->4       0.0
2->5      20.0
3->4      15.0
3->5       0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  This is separate from the ipykernel package so we can avoid doing imports until
